## Sympy and Widgets

Sympy is a google project to build a module for symbolic mathematics (think Mathematica, Maple, Maxima etc.). It supports

* Basic arithmetic: *, /, +, -, **
* Simplification
* Expansion
* Functions: trigonometric, hyperbolic, exponential, roots, logarithms, absolute value, spherical harmonics, factorials and gamma functions, zeta functions, polynomials, hypergeometric, special functions, ...
* Substitution
* Arbitrary precision integers, rationals and floats
* Noncommutative symbols
* Pattern matching

and it has modules for dealing with calculus, polynomials, discrete math, equation solving, geometry, combinartorics and much more. While not strictly a numerical package, it is increasingly used in other modules to provide the mathematical thinking behind generating _efficent_ instructions for numerical packages like numpy.

In the notebook we have access to great rendering capabilities so we can get pretty nice mathematical output from sympy just by asking...

In [ ]:
import sympy as sy
from sympy import init_printing

init_printing()

Sympy normally operates on 'symbols', which you can bind to ordinary python variables, they are simple to generate

In [ ]:
from sympy import symbols

x, y = symbols('x y')
type(x)

In [ ]:
x * y

A lot of the functionality (e.g. functions like `sin`) are available at the top level of the namespace (have a look at `sy.<TAB>`). You'll commonly see people do `from sympy import *`. This makes everything available to you, but it pollutes your namespace so we'll stick to explicitly giving the module alias (`sy.sin` etc.)

In [ ]:
sy.cos(x)

In [ ]:
expr = sy.cos(x) - sy.sin(y)
expr

### Substitution

`expr` is now an expression and one common thing you will need to do with expressions is substitute values for your symbols. The `subs` method can do this for you, just pass tuples with the substitution values (or a list of tuples)

In [ ]:
expr.subs(x, 1)

In [ ]:
expr.subs(y, x)

In [ ]:
expr.subs([(x, 0), (y, sy.pi/4)])

If you really need a floating point number, you can use the evalf method

In [ ]:
expr.evalf(subs={x: sy.pi/3, y:3})

### Algebra

There are functions for expanding, factoring, simplifying, collecting terms etc.

In [ ]:
a = (x+3)*(x-2)
a.expand()

In [ ]:
sy.simplify(sy.sin(x)**2 + sy.cos(x)**2)

**EXERCISE**: 
 * Try simplifying
$$
\frac{x^3 - 2x^2 -5x + 6}{x^2-4x+3}
$$
 * Use `.factor()` to factorize
$$
2x^4+x^3 - 33x^2 - 16x +16
$$

In [ ]:
expr2 = (x**2-1) / (x+1)
expr2

In [ ]:
expr2.cancel()

`simplify()` will miss some simplifications, but there is [extensive documentation](https://docs.sympy.org/latest/tutorial/simplification.html) on giving it a hand.

## Solving Simultaneous Equations

Sympy as a `solveset` routine which can help you simple equations. One gotcha with this is that you might expect to write an equation as e.g. `x**2 + 2x == -1`, but this will evaluate to False (try it). To get around this sympy as an `Eq` function for equality (an alternative is to rearrange things so that your equations are equal to zero (e.g `x**2 + 2*x - 1`)

In [ ]:
sy.Eq(x**2 + 2*x, -1)

To use solveset, just pass it an equation and (optionally) the variable you want to solve for

In [ ]:
sy.solveset(sy.Eq(x**2,-1), x)

So there are two solutions, $x=i$, & $x=-i$. By passing the `domain=` argument you can restrict solutions to a specific domain (e.g. $\mathbb{R}$ or $\mathbb{C}$)

For systems of linear equations there is linsolve which takes a list of equations and a tuple of the variables to solve for

In [ ]:
sy.linsolve([sy.Eq(x + 3 * y, 1), sy.Eq(y, 3)], (x, y))

In [ ]:
sy.solveset(sy.Eq(x**2,-1), x, domain=sy.Reals)

## Calculus

### Derivatives

sympy has lots of features for doing calculus. For simple differentiation sympy expressions have a `.diff()` method. You pass the variables you need to differentiate with respect to.

In [ ]:
sy.cos(x).diff(x)

**TODO**: Try differentiating
$$
\frac{(1 + 2x^2)(x^3-x)^2}{\sqrt{x+5x^2}}
$$
Hint: You'll need `sy.sqrt()`

There is a Derivative object which will let you create a derivative but delay evaluating it (this is more useful that it sounds, sometimes you'll want to massage the resulting expressions a bit before evaluating).

In [ ]:
expr2 = sy.Derivative(sy.cos(x), x)
expr2

When you're ready you can `doit()`

In [ ]:
expr2.doit()

### Integrals

Integrals are much harder to deal with systematically, but sympy does a pretty good job

In [ ]:
sy.integrate(expr, x)

You can also evaluate definite integrals, `oo` is sympy's notation for $\infty$

In [ ]:
sy.integrate(sy.exp(-x), (x, 0, 'oo'))

You can do multidimensional integrals with the same syntax we used to do multiple derrivatives. Similar to the `Derivative` object, we can delay the evaluation of the integral with the `Integral` object.

In [ ]:
expr3 = sy.Integral(sy.exp(-x**2-y**2), (x, 0, 'oo'), (y, 0, 'oo'))
expr3

In [ ]:
expr3.doit()

If sympy doesn't know how to evaluate an integral it will just return an `Integral` object which you can then work with to try to get it to evaluate, e.g. you can add assumptions on variable domains etc.

## Series

For numerical a fairly common pattern is to have some expression which you can't evaluate exactly, but which you can approximate with some finite number of terms. Function objects in sympy have a series method which will let you calculate a taylor series expansion of the function

In [ ]:
sxx = (sy.sin(x)/x).series(x, x0=0, n=12)
sxx

This is accurate up to $O(x^{12})$. You can extract individual terms you might need

In [ ]:
sxx.coeff(x**6).as_terms()

In [ ]:
import sympy as sy
x = sy.symbols('x')
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display, clear_output
from ipywidgets import widgets

def ftaylor(n):
    return (sy.sin(x)/x).series(x, x0=0, n=n)

fresult = interact(ftaylor, n=widgets.IntSlider(min=2,max=20,step=1,value=4));

So, we do the same thing visually with matplotlib, looking at how good our series approximation is compared to the exact value

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

def taylorApprx(n):
    clear_output(wait=True)
    func = sy.sin(x)/x
    taylor = sy.series(func, n=n).removeO()
    
    evalfunc = sy.lambdify(x, func, modules=['numpy'])
    evaltaylor = sy.lambdify(x, taylor, modules=['numpy'])
    
    t = np.linspace(-5.5, 5.5, 100)
    fig, ax = plt.subplots(1, figsize=(16,10))
    ax.plot(t, evalfunc(t), 'b', label='sin(x)/x')
    ax.plot(t, evaltaylor(t), 'r', label='{0} order appx.'.format(n))
    ax.legend(loc='best')
    sy.latex(taylor)
    plt.show()
    return taylor

a=widgets.interactive(taylorApprx, n=widgets.IntSlider(min=3,max=20,step=1,value=4))
display(a)